Con Document AI, podemos procesar documentos de diversos formatos y extraer información tanto de párrafos con mucho texto como de imágenes que lo contienen, como logotipos, texto manuscrito (firmas) o marcas de verificación. En este lección veremos la configuración de los objetos y privilegios necesarios, y la creación de un modelo de Document AI para su uso en un pipeline de procesamiento.

### Crear una base de datos y un rol

A continuación crearemos el esquema correspondiente a esta sección del curso y crearemos un rol personalizado `doc_ai_role` para crear el modelo de Document AI y crear un pipeline de procesamiento.

In [ ]:
-- Crear el esquema
CREATE OR REPLACE SCHEMA curso_ia.seccion_5;

In [ ]:
-- Crear un rol
CREATE OR REPLACE ROLE doc_ai_role;

### Privilegios necesarios

Para otorgar los privilegios necesarios para trabajar con Document AI, sigamos los siguientes pasos:

1. Otorgue el rol de base de datos `SNOWFLAKE.DOCUMENT_INTELLIGENCE_CREATOR` al rol `doc_ai_role`:

In [ ]:
GRANT DATABASE ROLE SNOWFLAKE.DOCUMENT_INTELLIGENCE_CREATOR TO ROLE doc_ai_role;

2. Otorgue privilegios de uso y operación del warehouse al rol `doc_ai_role`:


In [ ]:
GRANT USAGE, OPERATE ON WAREHOUSE compute_wh TO ROLE doc_ai_role;

3. Otorgue los privilegios para utilizar la base de datos y el esquema que creó al `doc_ai_role`:

In [ ]:
GRANT USAGE ON DATABASE curso_ia TO ROLE doc_ai_role;
GRANT USAGE ON SCHEMA curso_ia.seccion_5 TO ROLE doc_ai_role;

4. Otorgue el privilegio de creación de stage en el esquema al rol `doc_ai_role` para almacenar los documentos para su extracción:

In [ ]:
GRANT CREATE STAGE ON SCHEMA curso_ia.seccion_5 TO ROLE doc_ai_role;

5. Otorgue los privilegios para crear model builds (instancias de la clase DOCUMENT_INTELLIGENCE) al rol `doc_ai_role`:

In [ ]:
GRANT CREATE SNOWFLAKE.ML.DOCUMENT_INTELLIGENCE ON SCHEMA curso_ia.seccion_5 TO ROLE doc_ai_role;
GRANT CREATE MODEL ON SCHEMA curso_ia.seccion_5 TO ROLE doc_ai_role;

6. Otorgue el rol `doc_ai_role` a su usuario para su uso en el resto de la lección:


In [ ]:
-- Sustituya el usuario jose por su usuario en esta celda de código
GRANT ROLE doc_ai_role TO USER jose;
GRANT ROLE doc_ai_role TO ROLE ACCOUNTADMIN;

### Lista de entidades y preguntas

Las siguientes entidades y sus respectivas preguntas deberán llenarse en la construcción del modelo. Las preguntas están in Inglés para obtener mejores resultados debido a que los documentos están en Inglés.

- Ir a build details ⇒ Define values ⇒ Add entity
    - `inspection_date`: What is the inspection date?
    - `inspection_grade`: What is the grade?
    - `inspector`: Who performed the inspection?
    - `list_of_units`: What are all the units?

### Crear un pipeline de procesamiento de documentos

A continuación crearemos un pipeline de procesamiento utilizando el modelo de Document AI, los strems y las taks de Snowflake. El pipeline extraerá información de los nuevos documentos de inspección almacenados en un internal stage.
1. Cree un internal stage `pdf_stage` para almacenar los documentos:

In [ ]:
CREATE OR REPLACE STAGE curso_ia.seccion_5.pdf_stage
  DIRECTORY = (ENABLE = TRUE)
  ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE');

2. Cree un stream `pdf_stream` sobre el stage `pdf_stage`:


In [ ]:
CREATE OR REPLACE STREAM curso_ia.seccion_5.pdf_stream ON STAGE curso_ia.seccion_5.pdf_stage;

3. Actualice los metadatos de la tabla de directorio que almacenará los archivos de documentos preparados:

In [ ]:
ALTER STAGE curso_ia.seccion_5.pdf_stage REFRESH;

4. Cree una tabla `pdf_reviews` para almacenar la información sobre los documentos (como el nombre del archivo) y los datos que se extraerán de los documentos PDF:

In [ ]:
CREATE OR REPLACE TABLE curso_ia.seccion_5.pdf_reviews (
  file_name VARCHAR,
  file_size VARIANT,
  last_modified VARCHAR,
  snowflake_file_url VARCHAR,
  json_content VARCHAR
);

5. Cree una tarea `data_review` para procesar nuevos documentos en el stage:

In [ ]:
CREATE OR REPLACE TASK curso_ia.seccion_5.load_data_review
  WAREHOUSE = 'compute_wh'
  SCHEDULE = '1 minute'
  COMMENT = 'Procesa nuevos archivos en el stage e inserta datos en la tabla pdf_reviews.'
WHEN SYSTEM$STREAM_HAS_DATA('curso_ia.seccion_5.pdf_stream')
AS
INSERT INTO pdf_reviews (
  SELECT
    RELATIVE_PATH AS file_name,
    size AS file_size,
    last_modified,
    file_url AS snowflake_file_url,
    curso_ia.seccion_5.reviews!PREDICT(GET_PRESIGNED_URL('@curso_ia.seccion_5.pdf_stage', RELATIVE_PATH), 1) AS json_content
  FROM pdf_stream
  WHERE METADATA$ACTION = 'INSERT'
);

Tenga en cuenta que las tareas recién creadas se suspenden automáticamente.

6. Iniciar la tarea recién creada

In [ ]:
ALTER TASK curso_ia.seccion_5.load_data_review RESUME;

### Cargar documentos de inferencia al satge

En este paso usted deberá cargar los PDFs que se encuentran en la carpeta `datos_inferencia` adjunto como recurso a esta lección en el stage que hemos creado.

Luego de cargar los documentos esperamos a que la tarea se ejecute. Como la hemos calendarizado cada 1 minuto, no debe demorar en ejecutar.

Luego de que ejecute de forma correcta la tarea consultamos la tabla que creamos.

In [ ]:
SELECT * FROM curso_ia.seccion_5.pdf_reviews;

Si deseamos analizar la información extraída en columnas separadas podemos crear una nueva tabla como se muestra a continuación.

In [ ]:
CREATE OR REPLACE TABLE curso_ia.seccion_5.pdf_reviews_explode AS (
 WITH temp AS (
   SELECT
     RELATIVE_PATH AS file_name,
     size AS file_size,
     last_modified,
     file_url AS snowflake_file_url,
     curso_ia.seccion_5.reviews!PREDICT(get_presigned_url('@curso_ia.seccion_5.pdf_stage', RELATIVE_PATH), 1) AS json_content
   FROM directory(@curso_ia.seccion_5.pdf_stage)
 )

 SELECT
   file_name,
   file_size,
   last_modified,
   snowflake_file_url,
   json_content:__documentMetadata.ocrScore::FLOAT AS ocrScore,
   f.value:score::FLOAT AS inspection_date_score,
   f.value:value::STRING AS inspection_date_value,
   g.value:score::FLOAT AS inspection_grade_score,
   g.value:value::STRING AS inspection_grade_value,
   i.value:score::FLOAT AS inspector_score,
   i.value:value::STRING AS inspector_value,
   ARRAY_TO_STRING(ARRAY_AGG(j.value:value::STRING), ', ') AS list_of_units
 FROM temp,
   LATERAL FLATTEN(INPUT => json_content:inspection_date) f,
   LATERAL FLATTEN(INPUT => json_content:inspection_grade) g,
   LATERAL FLATTEN(INPUT => json_content:inspector) i,
   LATERAL FLATTEN(INPUT => json_content:list_of_units) j
 GROUP BY ALL
);

In [ ]:
SELECT * FROM curso_ia.seccion_5.pdf_reviews_explode;